In [7740]:
sample_size = 10

In [7754]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings("ignore")

In [7742]:
gene_df = pd.read_csv('data/gene_spec.csv')
morph_df = pd.read_csv('data/morph.csv')

In [7743]:
morph_df = morph_df.drop(['idx', 'Date', 'classification', 'uncertainty','Latitude', 'Longitude', 'Altitude.ft', 'Multi.Single.stem', 'General.location.Habitat', 'site', 'Putative_spp'], axis=1)
morph_df = morph_df.dropna()
morph_df = morph_df.replace(0, 1e-10)

data = morph_df.drop(['TreeNo'], axis=1)

scaler = StandardScaler()
data = scaler.fit_transform(data)
data = data[:, [0, 11, 7, 12, 16]]
kmeans = KMeans(n_clusters=3, random_state=0)
cluster_labels = kmeans.fit_predict(data)

morph_df['morph_cluster'] = cluster_labels
morph_df = morph_df[['TreeNo', 'morph_cluster']]

morph_df.to_csv('data/morph_cluster.csv', index=False)

In [7744]:
gene_df = pd.read_csv('data/gene_spec.csv')
gene_df = gene_df[gene_df['spec'] != 'QB']
gene_df = gene_df.drop(['DNA_ID', 'spec'], axis=1)
data = gene_df.drop(['TreeNo'], axis=1)

scaler = StandardScaler()
data = scaler.fit_transform(data)
data = data[:, [1,2,0]]
kmeans = KMeans(n_clusters=3, random_state=0)
cluster_labels = kmeans.fit_predict(data)

gene_df['gene_cluster'] = cluster_labels
gene_df.to_csv('data/gene_cluster.csv', index=False)

In [7745]:
joined_df= morph_df.merge(gene_df, on='TreeNo')

# sample a random batch of rows
sample = joined_df.sample(sample_size)

# build an association matrix
association_matrix = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        association_matrix[i,j] = np.sum((sample['morph_cluster'] == i) & (sample['gene_cluster'] == j))
print(association_matrix)

# Initialize decision array
decision = np.zeros(3, dtype=int)

# Create a copy of the association matrix to manipulate
temp_matrix = association_matrix.copy()

# Fill the decision array
for _ in range(3):
    # Find the index of the maximum value in the matrix
    max_index = np.argmax(temp_matrix)
    max_location = np.unravel_index(max_index, temp_matrix.shape)
    
    # Update the decision array with the column index of the maximum value
    decision[max_location[0]] = max_location[1]
    
    # Set the maximum value to a very low number to ignore it in future iterations
    temp_matrix[max_location[0], :] = -np.inf
    temp_matrix[:, max_location[1]] = -np.inf

print("Final decision array:")
print(decision)


[[0. 2. 2.]
 [0. 1. 3.]
 [1. 0. 1.]]
Final decision array:
[1 2 0]


In [7746]:
morph_df["morph_predicted_gene_cluster"] = morph_df.apply(lambda x: decision[x['morph_cluster']], axis=1)

In [7747]:
average_gene = gene_df.groupby('gene_cluster').agg({'PC1': 'mean', 'PC2': 'mean', 'PC3': 'mean'}).reset_index()
average_gene['gene_centroids'] = average_gene[['PC1', 'PC2', 'PC3']].values.tolist()
average_gene = average_gene[['gene_cluster', 'gene_centroids']]

In [7748]:
gene_df['gene_coordinates'] = gene_df[['PC1', 'PC2', 'PC3']].values.tolist()
gene_df = gene_df[['TreeNo', 'gene_coordinates']]
joined_df = morph_df.merge(gene_df, on='TreeNo').drop(['morph_cluster'], axis=1)
# rename columns
joined_df = joined_df.rename(columns={'morph_predicted_gene_cluster': 'gene_cluster'})
final_df = joined_df.merge(average_gene, on='gene_cluster')
final_df.head()

,TreeNo,gene_cluster,gene_coordinates,gene_centroids
0,QA-M11,0,"[-8.431936096, -0.500913737, -11.54222572]","[-6.734983830851852, -0.017621014611111093, -8.922407703351851]"
1,QA-M5,0,"[-8.593978553, 0.414067943, -16.68961594]","[-6.734983830851852, -0.017621014611111093, -8.922407703351851]"
2,QR-M1,2,"[19.47262912, -1.953619719, -2.119561558]","[16.470120757953126, -1.844546040984375, 1.384113856109375]"
3,QR-M14,0,"[22.37592773, -2.003944438, -2.581493893]","[-6.734983830851852, -0.017621014611111093, -8.922407703351851]"
4,QR-M15,0,"[22.73439088, -1.946043139, -1.961701937]","[-6.734983830851852, -0.017621014611111093, -8.922407703351851]"


In [7749]:
def euclidean_distance(row):
    return np.linalg.norm(np.array(row['gene_coordinates']) - np.array(row['gene_centroids']))

final_df['bkm_distance'] = final_df.apply(euclidean_distance, axis=1)

#### Control Group: KNearstNeighbors ####

In [7750]:
morph_df = pd.read_csv('data/morph.csv')
morph_df = morph_df.drop(['idx', 'Date', 'classification', 'uncertainty','Latitude', 'Longitude', 'Altitude.ft', 'Multi.Single.stem', 'General.location.Habitat', 'site', 'Putative_spp'], axis=1)
morph_df = morph_df.dropna()
morph_df = morph_df.replace(0, 1e-10)

data = morph_df.drop(['TreeNo'], axis=1)

scaler = StandardScaler()
data = scaler.fit_transform(data)
morph_df[["a", "b", "c", "d", "e"]] = data[:, [0, 11, 7, 12, 16]]

# convert to numpy matrix
morph_df['morph_coordinates'] = morph_df[['a', 'b', 'c', 'd', 'e']].values.tolist()

joined_df= morph_df.merge(gene_df, on='TreeNo')
joined_df = joined_df[['TreeNo', 'morph_coordinates', 'gene_coordinates']]

sample = joined_df.sample(sample_size)

In [7751]:
X = np.array(sample['morph_coordinates'].tolist())
y = np.array(sample['gene_coordinates'].tolist())

knn_regressor = KNeighborsRegressor(n_neighbors=3)
knn_regressor.fit(X, y)

morph_df['knn_gene_predictions'] = morph_df['morph_coordinates'].apply(lambda x: knn_regressor.predict([x])[0])
morph_df = morph_df[['TreeNo', 'knn_gene_predictions']]

final_df = morph_df.merge(final_df, on='TreeNo')

def euclidean_distance(row):
    return np.linalg.norm(np.array(row['gene_coordinates']) - np.array(row['knn_gene_predictions']))

final_df['knn_distance'] = final_df.apply(euclidean_distance, axis=1)

In [7752]:
# save the final_df
final_df.to_csv('data/final_df.csv', index=False)

In [7753]:
# on average, which distance is smaller?
print(final_df['bkm_distance'].mean())
print(final_df['knn_distance'].mean())

8.924153512625013
10.44868119067046
